In [47]:
!pip3 install transformers

In [113]:
from transformers import BertTokenizer, BertConfig, TFBertForSequenceClassification
import tensorflow as tf
import numpy as np
from tensorflow.keras import Input, layers, Model

# bert안에서 처리할 시퀀스의 최대 길이
MAX_SEQUENCE_LEN = 150

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
''' 네이버 영화 댓글 데이터 
import os

if not os.path.exists('./nsmc'):
    !git clone https://github.com/e9t/nsmc'''

" 네이버 영화 댓글 데이터 \nimport os\n\nif not os.path.exists('./nsmc'):\n    !git clone https://github.com/e9t/nsmc"

In [115]:
import pandas as pd

dict_from_csv = pd.read_csv('/content/여행지 테마별 분류_최종본(21.09.06).csv')
dict_from_csv.head()


,청정 우도에서 만난 청정 노을의 맑은 색,우도(해양도립공원),자연
0,보리밭을 지나다 아름다운 풍경에 홀릭 . 마침 남편의 생일을 맞아 멋진 사진을 남겨...,우도(해양도립공원),자연
1,보리밭을 지나다 아름다운 풍경에 홀릭 . 마침 남편의 생일을 맞아 멋진 사진을 남겨...,우도(해양도립공원),자연
2,우도섬에 봄이 왔습니다. 유채와 청보리 파란 하늘과 그보다 더 푸른섬 우도가 제주의...,우도(해양도립공원),자연
3,일몰이 제일 아름답다는 우도 낮에 사람들로 가득한 우도는 마지막 배가 떠나고 나면 ...,우도(해양도립공원),자연
4,세번째 제주여행 우도는 처음이였는데 너무 좋았어요,우도(해양도립공원),자연


In [116]:
dict_from_csv.columns=['text','name','label']

In [117]:
dict_from_csv.head()

,text,name,label
0,보리밭을 지나다 아름다운 풍경에 홀릭 . 마침 남편의 생일을 맞아 멋진 사진을 남겨...,우도(해양도립공원),자연
1,보리밭을 지나다 아름다운 풍경에 홀릭 . 마침 남편의 생일을 맞아 멋진 사진을 남겨...,우도(해양도립공원),자연
2,우도섬에 봄이 왔습니다. 유채와 청보리 파란 하늘과 그보다 더 푸른섬 우도가 제주의...,우도(해양도립공원),자연
3,일몰이 제일 아름답다는 우도 낮에 사람들로 가득한 우도는 마지막 배가 떠나고 나면 ...,우도(해양도립공원),자연
4,세번째 제주여행 우도는 처음이였는데 너무 좋았어요,우도(해양도립공원),자연


In [118]:
dict_from_csv=dict_from_csv.dropna(axis=0)

In [119]:
text_label=dict_from_csv[['text','label']]

In [ ]:
type(text_label['label'][0])

numpy.float64

In [120]:
my_text=text_label['text'].to_list()

In [121]:
my_label=text_label['label'].to_list()

In [ ]:
type(my_label)

list

In [ ]:
type(text_label)

pandas.core.frame.DataFrame

In [ ]:
text_label=text_label['label'].astype(int)

ValueError: ignored

In [ ]:
my_train_sentences={'text':my_text, 'label':my_label}

In [ ]:
type(my_train_sentences)

dict

In [ ]:
'''네이버 영화 댓글 데이터를 댓글과 라벨로 분리
with open('/content/nsmc/ratings_train.txt','r') as t:
  listt =[line.split('\t')  for line in t]
  text = [x[1] for x in listt]
  label = [x[2].replace('\n','') for x in listt]
  
  text = text[1:]
  label = label[1:]
  y_train = np.array([int(x) for x in label])

train_sentences = {'text': text, 'label':label}'''

In [ ]:
y_train=np.array([int(x) for x in my_label])

ValueError: ignored

In [122]:
from sklearn.preprocessing import LabelEncoder
encoder=LabelEncoder()
y=encoder.fit_transform(dict_from_csv['label'])

In [123]:
dict_from_csv['label']=y

In [124]:
y_train=dict_from_csv['label']

In [125]:
# 토크나이저 설정
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')

# 어휘집 생성
word2idx = tokenizer.get_vocab()
idx2word = {idx:word for idx, word in enumerate(word2idx)}

# 데이터 예시 확인
for idx in tokenizer.encode('뭐야 이 평점들은 나쁘진 않지만 100점 짜리는 더더욱 아니잖아'):
    print(idx2word[idx], end=' ')

[CLS] 뭐 ##야 이 평 ##점 ##들은 나 ##쁘 ##진 않 ##지만 100 ##점 짜 ##리는 더 ##더 ##욱 [UNK] [SEP] 

In [126]:
# 인코딩
train_encoded = tokenizer(my_text, padding=True, truncation=True, max_length=MAX_SEQUENCE_LEN, return_tensors='tf')

# 인코딩된 문장
X_train = train_encoded['input_ids']

# attention mask - pad masking용 attention
X_train_masks = train_encoded['attention_mask']

In [127]:
# 분류 모델 config 설정
my_config = BertConfig.from_pretrained(
    'bert-base-multilingual-cased',
    num_labels=3,
    output_hidden_states=False,      # 학습된 모든 레이어들의 hidden states의 반환 여부
    output_attentions=False,         # 학습된 모든 attention들의 반환 여부
    use_cache = False                # 마지막 Key/Value Attention의 반환 여부
)                                    # 개인적인 생각으로는 config의 모든 값들은 default로 사용해도 문제 없을 것 같네요

# 사전학습 모델 불러오기
bert_model = TFBertForSequenceClassification.from_pretrained('bert-base-multilingual-cased', config=my_config)
bert_model.trainable =False

All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at bert-base-multilingual-cased and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [128]:
# 모델 네트워크 설정 - TFBertForSequenceClassification 메소드가 tf.keras.Model을 상속받기 때문에 
# 저희가 배운 형태로 모델 설계가 가능합니다.
input_ids = Input(batch_shape=(None, MAX_SEQUENCE_LEN), dtype=tf.int32, name='input_ids')
input_masks = Input(batch_shape=(None, MAX_SEQUENCE_LEN), dtype=tf.int32, name='attention_masks')

embedding = bert_model(input_ids)[0] # logit값 반환

In [79]:
# 모델 네트워크 설정 - TFBertForSequenceClassification 메소드가 tf.keras.Model을 상속받기 때문에 
# 저희가 배운 형태로 모델 설계가 가능합니다.
input_ids = Input(batch_shape=(None, MAX_SEQUENCE_LEN), dtype=tf.int32, name='input_ids')
input_masks = Input(batch_shape=(None, MAX_SEQUENCE_LEN), dtype=tf.int32, name='attention_masks')

embedding = bert_model([input_ids, input_masks])[0] # logit값 반환

In [129]:
output = layers.Dense(3, activation='softmax')(embedding) # sigmoid: 이진 분류 -> softmax 3개 class

In [130]:
# 모델 구성 및 컴파일
model = Model(inputs=input_ids, outputs=output, name='Bert_Classification_1')
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
              loss='sparse_categorical_crossentropy', #다중 분류 모델이라서 손실 함수 변경
              metrics=['acc'])

In [81]:
# 모델 구성 및 컴파일
model = Model(inputs=[input_ids, input_masks], outputs=output, name='Bert_Classification_1')
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
              loss='sparse_categorical_crossentropy', #다중 분류 모델이라서 손실 함수 변경
              metrics=['acc'])

In [157]:
history = model.fit(x=X_train, y=y_train,
                 batch_size=512, validation_split=0.2,
                 epochs=2,shuffle=True)

Epoch 1/2
10/10 [==============================] - 82s 8s/step - loss: 1.1034 - acc: 0.2897 - val_loss: 1.0962 - val_acc: 0.4239
Epoch 2/2
10/10 [==============================] - 84s 9s/step - loss: 1.1022 - acc: 0.2973 - val_loss: 1.0971 - val_acc: 0.4115


In [83]:
history = model.fit(x=[X_train, X_train_masks], y=y_train,
                 batch_size=512, validation_split=0.3,
                 epochs=5,shuffle=True)

Epoch 1/5
10/10 [==============================] - 116s 12s/step - loss: 1.0859 - acc: 0.4744 - val_loss: 1.1422 - val_acc: 0.0247
Epoch 2/5
10/10 [==============================] - 116s 12s/step - loss: 1.0882 - acc: 0.4532 - val_loss: 1.1432 - val_acc: 0.0230
Epoch 3/5
10/10 [==============================] - 116s 12s/step - loss: 1.0880 - acc: 0.4630 - val_loss: 1.1442 - val_acc: 0.0230
Epoch 4/5
10/10 [==============================] - 117s 12s/step - loss: 1.0862 - acc: 0.4639 - val_loss: 1.1452 - val_acc: 0.0214
Epoch 5/5
10/10 [==============================] - 117s 12s/step - loss: 1.0893 - acc: 0.4513 - val_loss: 1.1463 - val_acc: 0.0206


In [158]:
test_data_set=pd.read_csv("/content/여행지_45개_테스트 데이터_test.csv",encoding='cp949')

In [159]:
test_data_set.head()

,성산 일출봉,자연,"푸른 바다 사이에 우뚝 솟은 성채와 같은 모양, 봉우리 정상에 있는 거대한 사발 모양의 분화구, 그리고 그 위에서 맞이하는 일출의 장관 때문에 성산일출봉은 많은 사람들의 감흥과 탄성을 자아낸다. 성산일출봉과 어우러진 경치는 아름답고 장관이다."
0,사려니숲길,자연,"사려니숲길은 제주의 숨은 비경 31곳 중 하나로, 비자림로를 시작으로 물찻오름과 사..."
1,협재 해수욕장,자연,투명한 물에 에메랄드빛 물감을 서서히 풀어놓은 듯한 바다빛은 보는 것만으로도 힐링이...
2,월정리 해변,자연,"아름다운 에메랄드빛 바다가 한 폭의 그림처럼 펼쳐져 있고, 그 위에는 밝은 달이 비..."
3,함덕 해수욕장,자연,"입구부터 환영해주는 키 큰 야자수들, 하얀 모래와 대조를 이루는 에메랄드빛 바다, ..."
4,산굼부리,자연,천연기념물 분화구이다. 이곳은 다양한 희귀식물들이 한 공간에 존재하는 '분화구 식물...


In [160]:
test_data_set.columns=['name','label','text']

In [161]:
y=encoder.transform(test_data_set['label'])

In [162]:
test_data_set['label']=y

In [163]:
my_text_1=test_data_set['text'].to_list()
my_label_1=test_data_set['label'].to_list()


y_test=test_data_set['label']

In [164]:
# 인코딩
test_encoded = tokenizer(my_text_1, padding=True, truncation=True, max_length=MAX_SEQUENCE_LEN, return_tensors='tf')

# 인코딩된 문장
X_test = test_encoded['input_ids']

X_test_masks = test_encoded['attention_mask']

In [165]:
loss, acc = model.evaluate(x=X_test, y = y_test)

2/2 [==============================] - 1s 184ms/step - loss: 1.0930 - acc: 0.4773


In [140]:
pre=model.predict(x=X_test)
pre

array([[0.3331525 , 0.33126286, 0.33558464],
       [0.33363795, 0.32984748, 0.33651456],
       [0.3286995 , 0.33310014, 0.3382003 ],
       [0.31551412, 0.33760297, 0.3468829 ],
       [0.32364997, 0.33537355, 0.34097648],
       [0.33622897, 0.3318464 , 0.33192465],
       [0.33654726, 0.32895738, 0.33449537],
       [0.34330508, 0.33067384, 0.32602105],
       [0.34070736, 0.33013335, 0.32915926],
       [0.34127685, 0.32988045, 0.32884273],
       [0.339133  , 0.33368227, 0.32718474],
       [0.34057468, 0.3301567 , 0.3292686 ],
       [0.3307746 , 0.33144122, 0.33778423],
       [0.3298376 , 0.3310262 , 0.3391362 ],
       [0.34325847, 0.33035058, 0.32639092],
       [0.34013274, 0.3312128 , 0.32865447],
       [0.34387434, 0.33045387, 0.32567182],
       [0.33603057, 0.3302408 , 0.3337287 ],
       [0.3363732 , 0.3348522 , 0.3287746 ],
       [0.34501332, 0.33153072, 0.323456  ],
       [0.3454402 , 0.33137128, 0.3231885 ],
       [0.34232613, 0.33198822, 0.32568568],
       [0.

In [111]:
loss, acc = model.evaluate(x=[X_test,X_test_masks], y = y_test)

2/2 [==============================] - 1s 222ms/step - loss: 1.1062 - acc: 0.3182


In [112]:
pre=model.predict(x=[X_test,X_test_masks])
pre

array([[0.3087333 , 0.3323418 , 0.35892487],
       [0.3151375 , 0.3340761 , 0.35078642],
       [0.31668487, 0.33020929, 0.35310587],
       [0.3135664 , 0.32886854, 0.35756508],
       [0.30926964, 0.3395127 , 0.3512177 ],
       [0.31473362, 0.33527943, 0.34998697],
       [0.32041347, 0.32550034, 0.35408616],
       [0.30864522, 0.33379295, 0.35756183],
       [0.3143    , 0.33102426, 0.3546757 ],
       [0.30630004, 0.33403483, 0.3596652 ],
       [0.3169337 , 0.33051232, 0.352554  ],
       [0.3215223 , 0.33063573, 0.34784195],
       [0.31598783, 0.33022088, 0.3537913 ],
       [0.30842137, 0.33680406, 0.35477456],
       [0.30461898, 0.33537337, 0.36000767],
       [0.30315417, 0.33615807, 0.36068773],
       [0.31274873, 0.33099142, 0.35625985],
       [0.31316775, 0.3364734 , 0.35035887],
       [0.314339  , 0.33336926, 0.3522917 ],
       [0.29635584, 0.34073037, 0.3629138 ],
       [0.29174814, 0.34457117, 0.36368066],
       [0.3055133 , 0.33602443, 0.35846224],
       [0.

In [ ]:
from tensorflow.keras.preprocessing import sequence

In [ ]:
# softmax 값 뽑아내는 코드 참고용

def infer(x):
    temp = tokenizer(x, padding=True, truncation=True, max_length=100, return_tensors='tf')
    temp_ids = sequence.pad_sequences(temp['input_ids'], 100)
    temp_mask = sequence.pad_sequences(temp['attention_mask'], 100)

    return tf.keras.activations.sigmoid(
        model([temp_ids, temp_mask]
    ))